# 3.6.1. Modelos Gaussianos Mixtos

## Preparación del Entorno

### Carga de Módulos

In [1]:
import math
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
#import session_info
from time import time
from plotly.subplots import make_subplots
from sklearn import set_config
from sklearn.datasets import make_blobs

# Tema Principal
from sklearn.mixture import GaussianMixture
import sys

#sys.path.append('../scripts')
from funny_stuffs import score_plot

### Configuración Inicial

In [2]:
random_seed = 333  # Semilla para reproducibilidad de resultados
np.random.seed(random_seed)  # Para reproducibilidad

# Configuración de opciones de visualización para pandas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', 15)  # Ajusta el número de filas a mostrar

In [ ]:
# Configuraciones extras
sns.set_style('dark')
dark_template = pio.templates['plotly_dark'].to_plotly_json()
dark_template['layout']['paper_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
dark_template['layout']['plot_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
pio.templates['plotly_dark_semi_transparent'] = go.layout.Template(dark_template)
pio.templates.default = 'plotly_dark_semi_transparent'
set_config(transform_output="pandas")
set_config(display='diagram')
warnings.filterwarnings("ignore")
%matplotlib inline

## Modelos Gaussianos Mixtos

### Fundamento Teórico

#### 1. **Definición**

Un GMM es una suma ponderada de $M$ componentes gaussianos (previamente definidos), donde cada componente contribuye a la densidad total con un peso específico. Matemáticamente, la densidad de probabilidad de un GMM se define como:

$$
p(\mathbf{x}|\Theta) = \sum_{i=1}^{M} \pi_i \mathcal{N}(\mathbf{x} | \mathbf{\mu}_i, \mathbf{\Sigma}_i)
$$

donde:
- $\mathbf{x}$ es un vector de datos.
- $\Theta$ representa los parámetros del modelo, que incluyen los pesos $\pi_i$, las medias $\mathbf{\mu}_i$ y las covarianzas $\mathbf{\Sigma}_i$ de cada componente gaussiano.
- $M$ es el número de componentes gaussianos en la mezcla.
- $\pi_i$ es el peso del $i$-ésimo componente gaussiano, donde $0 \leq \pi_i \leq 1$ y $\sum_{i=1}^{M} \pi_i = 1$.
- $\mathcal{N}(\mathbf{x} | \mathbf{\mu}_i, \mathbf{\Sigma}_i)$ es la densidad de probabilidad de una distribución gaussiana multivariante para el $i$-ésimo componente con media $\mathbf{\mu}_i$ y matriz de covarianza $\mathbf{\Sigma}_i$.


<details>
<summary> Distribución Gaussiana Multivariante de dimensión d </summary>

$$
\mathcal{N}(\mathbf{x} | \mathbf{\mu}, \mathbf{\Sigma}) = \frac{1}{(2\pi)^{d/2}|\mathbf{\Sigma}|^{1/2}} \exp\left(-\frac{1}{2}(\mathbf{x}-\mathbf{\mu})^T\mathbf{\Sigma}^{-1}(\mathbf{x}-\mathbf{\mu})\right)
$$


In [3]:
#Grupos con Distribución Gaussiana - N2
X, y_true = make_blobs(n_samples=300, centers=3, cluster_std=0.60, random_state=random_seed, center_box=(2, 2.5))

gmm = GaussianMixture(n_components=3, covariance_type='full').fit(X)
colors = ['darkred', 'darkgreen', 'darkblue']
labels = gmm.predict(X)
probs = gmm.predict_proba(X)

fig = go.Figure()
for i in range(3):
      fig.add_trace(go.Scatter3d(x=X[labels == i, 0], y=X[labels == i, 1], z=probs[labels == i, i],
                                    mode='markers', marker=dict(color=colors[i], size=5),
                                    name=f'Grupo {i+1}'))

for i, (mean, covar) in enumerate(zip(gmm.means_, gmm.covariances_)):
      # Generar una malla de puntos alrededor del centro de cada gaussiana
      x, y = np.meshgrid(np.linspace(-3, 6), np.linspace(-3, 8))
      z = np.dstack((x, y))
      kernel = np.linalg.det(2 * np.pi * covar) ** -0.5
      inv_covar = np.linalg.inv(covar)
      mahalanobis = np.sum((z - mean) @ inv_covar * (z - mean), axis=2)
      gaussian_surface = np.exp(-0.5 * mahalanobis) * kernel
      fig.add_trace(go.Surface(z=gaussian_surface, x=x, y=y, opacity=0.5, 
                              showscale=False, colorscale=[[0, colors[i]], [1, colors[i]]]))

      fig.update_layout(title='Grupos con Distribución Gaussiana',
                  scene=dict(
                        xaxis_title='X',
                        yaxis_title='Y',
                        zaxis_title='Probabilidad'),
                  legend_title="Grupos",
                  height=500, width=1200)

fig.show()

#### 2. **Estimación de Parámetros**

La estimación de los parámetros $\Theta = \{\pi_i, \mathbf{\mu}_i, \mathbf{\Sigma}_i\}$ se realiza comúnmente a través del algoritmo Expectation-Maximization (EM), que busca maximizar la verosimilitud del modelo dado los datos observados. El algoritmo EM tiene dos pasos principales que se repiten iterativamente:

**Inicialización:**

Establecer valores iniciales para $\mu_k$, $\Sigma_k$, y $\pi_k$.

**Paso $E$ (Expectation):**

Calcula la probabilidad posterior de que cada dato provenga de un determinado componente gaussiano, utilizando los parámetros actuales del modelo. Esto se hace calculando la responsabilidad $\gamma(z_{ik})$ de cada componente $i$ para cada punto de datos $\mathbf{x}_k$:

$$
\gamma(z_{ik}) = \frac{\pi_i \mathcal{N}(\mathbf{x}_k | \mathbf{\mu}_i, \mathbf{\Sigma}_i)}{\sum_{j=1}^{M} \pi_j \mathcal{N}(\mathbf{x}_k | \mathbf{\mu}_j, \mathbf{\Sigma}_j)}
$$

donde:
- $\gamma(z_{ik})$ es la responsabilidad que el componente $i$ tiene por la observación $\mathbf{x}_k$.
- $\pi_i$ es el peso del $i$-ésimo componente gaussiano.
- $\mathcal{N}(\mathbf{x}_k | \mathbf{\mu}_i, \mathbf{\Sigma}_i)$ es la densidad de probabilidad de que la observación $\mathbf{x}_k$ sea generada por el $i$-ésimo componente gaussiano, dados su media $\mathbf{\mu}_i$ y matriz de covarianza $\mathbf{\Sigma}_i$.

**Paso $M$ (Maximization):**

Actualiza los parámetros del modelo $\Theta$ para maximizar la verosimilitud esperada obtenida en el Paso E. Los nuevos parámetros se calculan como sigue:

- Nuevas medias:
$$
\mathbf{\mu}_i^{nuevo} = \frac{1}{N_i} \sum_{k=1}^{N} \gamma(z_{ik}) \mathbf{x}_k
$$
- Nuevas covarianzas:
$$
\mathbf{\Sigma}_i^{nuevo} = \frac{1}{N_i} \sum_{k=1}^{N} \gamma(z_{ik}) (\mathbf{x}_k - \mathbf{\mu}_i^{nuevo})(\mathbf{x}_k - \mathbf{\mu}_i^{nuevo})^T
$$
- Nuevos pesos:
$$
\pi_i^{nuevo} = \frac{N_i}{N}
$$
donde $N_i = \sum_{k=1}^{N} \gamma(z_{ik})$ es el número efectivo de puntos asignados al componente $i$, y $N$ es el número total de puntos de datos.

<details>
<summary> ¿De donde salen los nuevos parámetros? </summary>

Las fórmulas para los nuevos parámetros se derivan maximizando la función de verosimilitud esperada bajo la restricción de que los pesos sumen uno. Este proceso utiliza el método de los multiplicadores de Lagrange:


El objetivo de la maximización es maximizar la función de verosimilitud esperada $Q(\Theta, \Theta^{(t)})$ con respecto a los parámetros $\Theta = \{\pi_i, \mathbf{\mu}_i, \mathbf{\Sigma}_i\}$, donde $t$ indica la iteración actual del algoritmo. La función $Q$ se define como la expectativa de la log-verosimilitud bajo la distribución posterior de los datos latentes $Z$ dada una observación $\mathbf{X}$ y los parámetros actuales $\Theta^{(t)}$.


Para actualizar cada pámetro $\mathbf{param}_i$, tomamos la derivada de $Q$ con respecto a $\mathbf{param}_i$ y la igualamos a cero. Esto implica derivar la expectativa de la log-verosimilitud con respecto a $\mathbf{param}_i$ y resolver para $\mathbf{param}_i$ cómo en sus añoradas clases de CIV.

$$
\frac{\partial Q}{\partial \mathbf{\mu}_i} = 0 \Rightarrow \mathbf{\mu}_i^{nuevo}
$$

$$
\frac{\partial Q}{\partial \mathbf{\Sigma}_i} = 0 \Rightarrow \mathbf{\Sigma}_i^{nuevo}
$$

$$
\pi_i^{nuevo} = \frac{N_i}{N}
$$

donde $N$ es el número total de observaciones, asegurando que los nuevos pesos también sumen uno.

#### 3. **Convergencia**

El algoritmo EM alterna entre los pasos E y M hasta alcanzar la convergencia. Determinar dicha convergencia es observar el cambio en la log-verosimilitud del modelo entre iteraciones consecutivas:

$$
\log p(\mathbf{X} | \Theta) = \log \left(\sum_{i=1}^{M} \pi_i \mathcal{N}(\mathbf{X} | \mathbf{\mu}_i, \mathbf{\Sigma}_i)\right)
$$

Si el cambio en la log-verosimilitud es menor que un umbral predeterminado, el algoritmo ha convergido:

$$
\left| \log p(\mathbf{X} | \Theta^{(t)}) - \log p(\mathbf{X} | \Theta^{(t-1)}) \right| < \epsilon
$$

donde:
- $ \Theta^{(t)} $ y $ \Theta^{(t-1)} $ son los parámetros del modelo en la iteración actual $ t $ y en la iteración anterior $ t-1 $, respectivamente.
- $ \epsilon $ es un umbral pequeño positivo predefinido.


#### Aplicaciónes y Consideraciones

<p style="font-size:25px;">Aplicaciones</p>

1. Biología Computacional y Genómica
2. Análisis de Redes Sociales
3. Segmentación de Clientes
4. Análisis de Documentos y Textos
5. Recomendadores


<p style="font-size:25px;">Ventajas</p>

1. Modelado de Complejidad
2. Estimación de Densidades
3. Asignación Blanda de Clusters
4. Incorporación de Incertidumbre


<p style="font-size:25px;">Consideraciones</p>

1. Elección del Número de Componentes
2. Inicialización `(k-means, random)`
3. Dependencia de la Forma de los Clusters
4. Sensibilidad a Datos Atípicos
5. Costo Computacional
6. Mínimos Locales
7. Interpretación de Componentes
8. Necesidad de Normalización
9. Evaluación del Modelo

### Ejemplo Práctico

#### Preparación de los Datos

**Objetivo**

El objetivo es clasificar diferentes especies de pingüinos en grupos naturales basándonos en características morfológicas y físicas, como la longitud del pico, profundidad del pico, longitud de las aletas, y peso, para entender mejor las relaciones entre las diferentes especies y facilitar su identificación y estudio.

**Planteamiento del problema**

Se ha reunido un conjunto de datos con muestras de pingüinos recogidas de diversas localizaciones y condiciones ambientales en la región del Antártico y subantártico. El equipo de investigación necesita una forma de categorizar las muestras eficientemente para apoyar estudios evolutivos, ecológicos y de conservación. Sin etiquetas preexistentes para las especies, necesitan un método de agrupamiento que organice los pingüinos en grupos significativos basados en sus características físicas.

Las características extraídas de cada elemento son:

1. **Longitud del Pico (Beak Length)**: La longitud del pico en milímetros. El pico de los pingüinos es crucial para su supervivencia, ya que lo utilizan para capturar comida y en rituales de apareamiento.

2. **Profundidad del Pico (Beak Depth)**: La profundidad del pico en milímetros. Esta medida se toma en la parte más profunda del pico y es un indicador importante de la dieta y el nicho ecológico de la especie.

3. **Longitud de las Aletas (Flipper Length)**: La longitud de las aletas en milímetros. Las aletas de los pingüinos son vitales para la natación y la búsqueda de alimento en el agua.

4. **Peso del Cuerpo (Body Mass)**: El peso del cuerpo en gramos. El peso puede indicar el estado de salud, la madurez y la capacidad reproductiva del pingüino.

Estas medidas se utilizarán para agrupar a los pingüinos en categorías que reflejen diferencias y similitudes evolutivas y adaptativas, lo que ayudará en la conservación y el estudio de estas especies en su hábitat natural.

In [4]:
# Carga del conjunto de datos
df = pd.read_csv('../data/penguins.csv')

In [5]:
df.species.unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [6]:
# Grupos Reales:
y = df['species']

df.drop(['species','sex','island'], axis=1, inplace=True)

In [7]:
df.sample(5)

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
217,49.6,18.2,193.0,3775.0
22,35.9,19.2,189.0,3800.0
197,50.8,18.5,201.0,4450.0
32,39.5,17.8,188.0,3300.0
249,50.0,15.3,220.0,5550.0


In [8]:
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [9]:
df = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].dropna()

fig = px.scatter_matrix(df,
                        dimensions=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
                        title="Pairplot de Características",
                        opacity=0.5,
                        height=500, width=1200)

for i in range(len(fig.data)):
    fig.data[i].showupperhalf = False
    fig.data[i].diagonal.visible = False
    fig.data[i].marker.size = 3
fig.show()

Aquí difusamente podemos notar grupos esféricos y en algunos cruces podemos ver tres grupos independientes.

#### Implementación del Método

Principales parámetros de $GaussianMixture$:
```python
GaussianMixture(
    n_components=1,
    covariance_type='full',
    tol=0.001,
    max_iter=100,
    init_params='kmeans',
)

#### Visualización e Implementación de Resultados

In [10]:
# Rango de número de clusters a evaluar
n_clusters = np.arange(2, 6)
bic_scores = []; aic_scores = []

# Ajustar GMM para cada número de clusters y calcular BIC y AIC
for n in n_clusters:
    gmm = GaussianMixture(n_components=n, random_state=random_seed).fit(df)
    bic_scores.append(gmm.bic(df))
    aic_scores.append(gmm.aic(df))

In [11]:
score_plot(scores=bic_scores, range_n_clusters=n_clusters, 
         operator='min', name_index = 'BIC')

In [12]:
score_plot(scores=aic_scores, range_n_clusters=n_clusters, 
         operator='min', name_index = 'AIC')

In [13]:
# Generamos GMM con 3 Clusters
gmm = GaussianMixture(n_components=3, random_state=random_seed).fit(df)

# Predecir los clusters
clusters = gmm.predict(df)

In [14]:
df['Grupos Reales'] = y
df['Grupos GMM'] = clusters

df['Grupos GMM'] = df['Grupos GMM'].replace({1: 'G1', 0: 'G2', 2: 'G3'})

In [15]:
confusion_matrix = pd.crosstab(df['Grupos Reales'], df['Grupos GMM'])

# Matriz de Confusión
fig = ff.create_annotated_heatmap(z=confusion_matrix.values,
                                 x=list(confusion_matrix.columns),
                                 y=list(confusion_matrix.index),
                                 colorscale='Greys')

fig.update_layout(title='Matriz de Confusión',
                  xaxis=dict(title='Grupos GMM'),
                  yaxis=dict(title='Grupos Reales'))

# Mostrar la figura
fig.show()

In [16]:
df[(df['Grupos Reales']=='Chinstrap')&(df['Grupos GMM']=='G3')]

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Grupos Reales,Grupos GMM
172,42.4,17.3,181.0,3600.0,Chinstrap,G3
182,40.9,16.6,187.0,3200.0,Chinstrap,G3
206,42.5,17.3,187.0,3350.0,Chinstrap,G3


In [17]:
selected_observations = df.loc[[172, 182, 206],
                              ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].copy()

In [18]:
pd.DataFrame(gmm.predict_proba(selected_observations).round(3), 
            index=selected_observations.index, 
            columns=['G2','G1','G3'])

,G2,G1,G3
172,0.064,0.0,0.936
182,0.219,0.0,0.781
206,0.456,0.0,0.544


**Conclusión**

Este ejercicio demostró cómo aplicar con éxito el modelo Gaussian Mixture Model (GMM) ya que tenemos por intuición el comportamiento de la distibución de las mediciónes de los pinguinos. Mediante este enfoque, se buscó identificar patrones subyacentes en los datos que permitieran agrupar a los pingüinos en distintas categorías (clusters) que pudieran corresponder a sus especies biológicas.